In [1]:
import pandas as pd
pd.options.display.max_columns = 150
#pd.set_option(max_columns,150)

In [2]:
import sqlite3 as sql

In [3]:
DB = "jobs.db"

In [4]:
conn =  sql.connect("jobs.db")

In [5]:
def run_query(q):
    return pd.read_sql_query(q,conn)

In [6]:
q = """SELECT COUNT(Major) , MIN(Median) 
       FROM recent_grads
       WHERE ShareWomen > 0.5"""
run_query(q)

,COUNT(Major),MIN(Median)
0,97,22000


In [7]:
q = """select name,type FROM sqlite_master WHERE type = 'table'"""
run_query(q)

,name,type
0,recent_grads,table


In [8]:
q = """select * FROM recent_grads limit 5 """
run_query(q)

,index,Rank,Major_code,Major,Major_category,Total,Sample_size,Men,Women,ShareWomen,Employed,Full_time,Part_time,Full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th,College_jobs,Non_college_jobs,Low_wage_jobs
0,0,1,2419,PETROLEUM ENGINEERING,Engineering,2339,36,2057,282,0.120564,1976,1849,270,1207,37,0.018381,110000,95000,125000,1534,364,193
1,1,2,2416,MINING AND MINERAL ENGINEERING,Engineering,756,7,679,77,0.101852,640,556,170,388,85,0.117241,75000,55000,90000,350,257,50
2,2,3,2415,METALLURGICAL ENGINEERING,Engineering,856,3,725,131,0.153037,648,558,133,340,16,0.024096,73000,50000,105000,456,176,0
3,3,4,2417,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,1258,16,1123,135,0.107313,758,1069,150,692,40,0.050125,70000,43000,80000,529,102,0
4,4,5,2405,CHEMICAL ENGINEERING,Engineering,32260,289,21239,11021,0.341631,25694,23170,5180,16697,1672,0.061098,65000,50000,75000,18314,4440,972


In [9]:
q = """select Major_category, AVG(ShareWomen) Avg_ShareWomen
      From recent_grads
      Group By Major_category
      Order By Avg_ShareWomen Desc"""
run_query(q)

,Major_category,Avg_ShareWomen
0,Psychology & Social Work,0.777763
1,Humanities & Liberal Arts,0.676193
2,Education,0.674986
3,Communications & Journalism,0.643835
4,Agriculture & Natural Resources,0.617938
5,Health,0.616857
6,Biology & Life Science,0.584518
7,Arts,0.561851
8,Social Science,0.539067
9,Computers & Mathematics,0.512752


In [10]:
q = """select Major_category, AVG(ShareWomen) Avg_ShareWomen
      From recent_grads
      Group By 1
      Order By 2 Desc
      limit 5"""
run_query(q)

,Major_category,Avg_ShareWomen
0,Psychology & Social Work,0.777763
1,Humanities & Liberal Arts,0.676193
2,Education,0.674986
3,Communications & Journalism,0.643835
4,Agriculture & Natural Resources,0.617938


In [11]:
q = """Select Major_category, Avg(Low_wage_jobs)/Avg(Total)  share_wage
      From recent_grads
      Group By 1
      Having share_wage > 0.1
      limit 5"""                        #having jb lgta ha jb group kia jae q k having apna column rkhta ha
run_query(q)

,Major_category,share_wage
0,Arts,0.168331
1,Communications & Journalism,0.126324
2,Humanities & Liberal Arts,0.132087
3,Industrial Arts & Consumer Services,0.115713
4,Law & Public Policy,0.115685


In [12]:
q = """select Major_category,CAST (Sum(Women) as float)/CAST(Sum(Total) as float) SW from recent_grads 
      Group By 1
      Having SW > 1
      Order By 2 Desc
      
      limit 5"""
run_query(q)          #cast dtype chnge krne k lie use hota ha

,Major_category,SW
0,Agriculture & Natural Resources,3.123392
1,Physical Sciences,1.320883
2,Biology & Life Science,1.273806
3,Education,1.096273
4,Psychology & Social Work,1.049178


In [20]:
 q = """select ShareWomen, Avg(ShareWomen) as avg
        from recent_grads
        where ShareWomen > Avg(ShareWomen)"""
run_query(q)

DatabaseError: Execution failed on sql 'select ShareWomen, Avg(ShareWomen) as avg
       from recent_grads
       where ShareWomen > Avg(ShareWomen)': misuse of aggregate: Avg()

In [21]:
q = """select ShareWomen 
      From recent_grads
      Where ShareWomen > (Select Avg(ShareWomen) from recent_grads)
      limit 5"""
run_query(q)

,ShareWomen
0,0.535714
1,0.578766
2,0.558548
3,0.896019
4,0.750473


In [23]:
q = """select count(ShareWomen) 
      From recent_grads
      Where ShareWomen > (Select Avg(ShareWomen) from recent_grads)
      """
run_query(q)

,count(ShareWomen)
0,91


In [24]:
q = """Select CAST(count(*) as float)/ (Select count(*) From recent_grads) propotion From recent_grads
       Where ShareWomen> (select Avg(ShareWomen) from recent_grads)"""
run_query(q)

,propotion
0,0.526012


In [26]:
q = """select Major,Major_category  
      from recent_grads
      Group By 2
      Order By sum(Total)
      limit 5"""
run_query(q)

,Major,Major_category
0,MULTI/INTERDISCIPLINARY STUDIES,Interdisciplinary
1,FOOD SCIENCE,Agriculture & Natural Resources
2,COURT REPORTING,Law & Public Policy
3,ASTRONOMY AND ASTROPHYSICS,Physical Sciences
4,CONSTRUCTION SERVICES,Industrial Arts & Consumer Services


In [33]:
q = """Select Major,Major_category,
       cast(Sample_size as float)/Total avg_ratio  
       From recent_grads
       Where  avg_ratio > (select Avg(cast(Sample_size as float)/Total) From recent_grads)
       limit 5"""
run_query(q)

,Major,Major_category,avg_ratio
0,PETROLEUM ENGINEERING,Engineering,0.015391
1,MINING AND MINERAL ENGINEERING,Engineering,0.009259
2,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,0.012719
3,ACTUARIAL SCIENCE,Business,0.013503
4,MECHANICAL ENGINEERING,Engineering,0.011280


In [72]:
q = """select Major,Major_category,Sample_size,Total,
       cast(Sample_size as float)/Total avg_ratio  
       From recent_grads
       group by 2
       having avg_ratio > (select Avg(cast(Sample_size as float)/Total) from recent_grads )
       limit 5
       """
run_query(q)

,Major,Major_category,Sample_size,Total,avg_ratio
0,ACTUARIAL SCIENCE,Business,51,3777,0.013503
1,COMMUNICATIONS,Communications & Journalism,2394,213996,0.011187
2,COMPUTER SCIENCE,Computers & Mathematics,1196,128319,0.009321
3,PETROLEUM ENGINEERING,Engineering,36,2339,0.015391
4,NURSING,Health,2554,209394,0.012197


In [65]:
q ="""select Sample_size,Total,
      cast(Sample_size as float)/Total avg_ratio
      From recent_grads
      where  avg_ratio > (select Avg(cast(Sample_size as float)/Total) from recent_grads)
      limit 5"""
run_query(q)

,Sample_size,Total,avg_ratio
0,36,2339,0.015391
1,7,756,0.009259
2,16,1258,0.012719
3,51,3777,0.013503
4,1029,91227,0.011280
